In [ ]:
%matplotlib notebook
from obspy import read
import numpy as np
import sys 
sys.path.append("/Users/Hilary/Documents/Github/Py-NnK/scan")
import os
import EQScanner
import EQ
import importlib
import matplotlib.pyplot as plt
import obspy
import trigger
import utm
import time
import winsound

In [ ]:
# Load Travel Time Model data from file
TTModel_P= np.load('save/Travel_Time_Model_P.npy')
TTModel_S= np.load('save/Travel_Time_Model_S.npy')
src_lalo=[63.90,-22.05]
src_z=6 
src_x=utm.from_latlon(63.90, -22.05)[0]/100
src_y=utm.from_latlon(63.90, -22.05)[1]/100
print('x={}, y={}'.format(src_x,src_y))

### Scan S

In [ ]:
for nst in range(0,18+1,2):
    no_of_event='Slow_P-1_S(p-1)178_random_n10_{}st'.format(nst) # Will be used for saving files.
    # Read synthetic steam from disk
    stm = read('save/Test_Stream_{}/{}_Trace_*'.format(no_of_event,no_of_event))

    # Process and trim the raw trace
    trim_start=5
    trim_end= -225
    stm_processed=EQScanner.signal_processing(stm=stm,
                                process=False,hp=5.0,lp=49.0, 
                                trim= True, trim_start= trim_start, trim_end= trim_end, 
                                plot=False)

    # For generating synthetic seismogram example (plot only 1 station)
    #stm_processed[:3].plot(size=(1200,400),outfile='save/figs/{}_syn_seis_ex.png'.format(no_of_event),format='png')

    for cf_type_l in [['CECM'],['ST*LT'], ['ST*LT','CECM'],['ST/LT'],['RP/LP'] ]:#['CECM'],['ST*LT'], ['ST*LT','CECM'], 
        cf_type_list =cf_type_l
        cf_type,cf_E,cf_N,cf_Z= EQScanner.chracteristic_function(
                                                    stm_processed, cf_type_list= cf_type_list, 
                                                    plot= True, plot_size=(10,14), save_fig=True,
                                                    save_cf= False, no_of_event=no_of_event,linewidth=1)
        
        #Cf_EN=np.sqrt(cf_E*cf_N)
        plt.close()
        time.sleep(1)

        # Parameters for scanning event origin time
        tmin,tmax = 23,26                     # Range of possible origin time (in sec after the start time)
        dt = 0.1                             # Step for scanning specific time (in sec)
        # Parameters for scanning the hypocenter
        zmin,zmax = src_z-6, src_z+6        # Range of possible depth (in km)
        xmin,xmax = src_x-60, src_x+60               # E-W boundaries of the scanning area 
                                             # (in Easting; omit the last 2 digits: ie. dx=100m)
        ymin,ymax = src_y-60, src_y+60             # S-N boundaries of the scanning area
                                             # (in Northing; omit the last 2 digits: ie. dy=100m)
        dx = 5                               # x interval (ex. dx = 5 then resolusion = 500m)
        dy = 5                               # y interval (ex. dy = 5 then resolusion = 500m)
        dz = 0.5                             # z interval (ex. dx = 0.5 then resolusion = 0.5km)
        utm_zone = (27,'V')                  # UTM zone number of the scanning area (ex. (27,'V))
        method  = 'L1'                  # Option:'L2-Norm', 'EDT'
        #method  = 'EDT'                     # Option:'L2-Norm', 'EDT'
        
        color_dict=  {'kri':'tab:blue','faf':'tab:orange','lat':'tab:green','sea':'tab:red',
                     'klv':'tab:purple','elb':'xkcd:sienna','ash':'tab:pink','iss':'tab:gray',
                     'hdv':'tab:olive','lsf':'tab:cyan','lhl':'xkcd:tomato','vog':'xkcd:teal',
                     'kas':'xkcd:green','blf':'xkcd:blue','bhr':'xkcd:orchid','hah':'xkcd:orangered',
                     'svh':'xkcd:khaki','has':'xkcd:grey'}
        
        # Scan to get the brightness matrices
        C_spatial, C_temporal    = EQScanner.SourceScan_Phase(   
                                    stm_processed= stm_processed, 
                                    Cf_Z=cf_Z, Cf_E=None, Cf_N=None, 
                                    tmin= tmin, tmax= tmax, dt= dt,
                                    zmin= zmin, zmax= zmax, dz= dz,
                                    xmin= xmin, xmax= xmax, dx= dx,
                                    ymin= ymin, ymax= ymax, dy= dy, 
                                    utm_zone= utm_zone, TTModel_1= TTModel_P, TTModel_2=None,
                                    method=method, tmin_checker = False, 
                                    save_C= True, no_of_event= no_of_event, cf_type_list=cf_type_list,
                                    add_filename='',alarm=False)

        C_s= C_spatial
        time.sleep(1)

        # Analyse to get the maximun brightness
        C_xy, C_zy, C_zx, coor_max_Cxy, coor_max_Czy, coor_max_Czx, imax_Cs= EQScanner.analyse_spatial(
                                    stm_processed= stm_processed, C_s= C_s,
                                    zmin= zmin, zmax= zmax, dz= dz,
                                    xmin= xmin, xmax= xmax, dx= dx,
                                    ymin= ymin, ymax= ymax, dy= dy)

        t_eo= EQScanner.analyse_temporal(
            stm_processed= stm_processed, C_temporal= C_temporal, imax_Cs=imax_Cs, tmin=tmin)

        # Plot the Source Scanning result
        EQScanner.source_plot(    stm_processed=stm_processed,
                        C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                        coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                        dt= dt, method=method,cf_type_list=cf_type_list,
                        zmin= zmin, zmax= zmax, dz= dz,
                        xmin= xmin, xmax= xmax, dx= dx,
                        ymin= ymin, ymax= ymax, dy= dy,
                        plot_mp = False,# mp_scale = 0.1, C_6D=C_6D,
                        draw_rect=False, 
                        #rect_x=[4465,4485], rect_y=[70850,70880], rect_z= [0,50],
                        plot_ref= '--', figsize=[7,7.3], vmin=None, 
                        compare_dtdr=True, 
                        tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                        save_fig=True, no_of_event= no_of_event, add_figname='_scanP_nosmth')
        plt.close()

        # Smooth the brightness matrix
        C_s= EQScanner.smooth(      C_spatial,
                                    zmin= zmin, zmax= zmax, dz= dz,
                                    xmin= xmin, xmax= xmax, dx= dx,
                                    ymin= ymin, ymax= ymax, dy= dy)

        # Analyse to get the maximun brightness
        C_xy, C_zy, C_zx, coor_max_Cxy, coor_max_Czy, coor_max_Czx, imax_Cs= EQScanner.analyse_spatial(
                                    stm_processed= stm_processed, C_s= C_s,
                                    zmin= zmin, zmax= zmax, dz= dz,
                                    xmin= xmin, xmax= xmax, dx= dx,
                                    ymin= ymin, ymax= ymax, dy= dy)

        t_eo= EQScanner.analyse_temporal(
            stm_processed= stm_processed, C_temporal= C_temporal, imax_Cs=imax_Cs, tmin=tmin)

        # Compare SSA result with SAC reference
        diff_in_t, diff_in_s = EQScanner.compare(	stm_processed=stm_processed,
                            C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                            coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                            tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                            save = True, no_of_event=no_of_event, method=method, add_title='_scanP_',
                            cf_type=cf_type,first_cf_type='CECM', filename='Results_slowPS__n10_scanP_random')
    
        # Plot the Source Scanning result
        EQScanner.source_plot(    stm_processed=stm_processed,
                        C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                        coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                        dt= dt, method=method,cf_type_list=cf_type_list,
                        zmin= zmin, zmax= zmax, dz= dz,
                        xmin= xmin, xmax= xmax, dx= dx,
                        ymin= ymin, ymax= ymax, dy= dy,
                        plot_mp = False, #, mp_scale=0.1, C_4D=C_4D,
                        compare_dtdr=True, 
                        tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                        draw_rect=False, 
                        #rect_x=[4465,4485], rect_y=[70850,70880], rect_z= [0,50],
                        plot_ref= '--', figsize=[7,7.3], vmin=None,
                        save_fig=True, no_of_event= no_of_event, add_figname='_scanP_smth')
        plt.close()
        time.sleep(1)
winsound.Beep(523, 1000)
winsound.Beep(587, 500)
winsound.Beep(523, 500)

In [ ]:
EQScanner.waveform_plot_all(stm_processed= stm_processed, duration= 12, channel='Z', 
                    TTModel= TTModel_P, TTModel_2=TTModel_S, 
                    trim_start= trim_start,
                    utm_zone= utm_zone, 
                    figsize=[8,11], xy_label_size=20,  # nst18:[8,11], nst10:[8,8], nst5:[8,5]
                    amp=0.5, color_dict=color_dict,
                    save_fig= False, no_of_event=no_of_event, 
                    method='SAC', cf_type_list=cf_type_list, add_figname='_scanPS_',
                    title_size=24, add_v_sep=0.05,
                    tt_marker='k+',tt_mk_size=6,tt_marker_sep=0.2) #syn:26,real: 6
#waveform_plot(stm_processed, duration, 
#        TTModel, 
#        trim_start=None,
#        tmin=None, t_eo=None,  utm_zone=None, 
#        channel='Z',figsize=[7,11],amp=0.6,color_dict=None, TTModel_2=None, 
#        save_fig=False, no_of_event= None,cf_type_list=None,method =None, add_figname='_scanPS_',
#        title_size=24, xy_label_size=20,add_v_sep=0,
#        tt_marker='k+',tt_mk_size=6,tt_marker_sep=0.2):

### Scan P and S

In [ ]:
for nst in range(0,18+1,2):
    no_of_event='Slow_P-1_S(p-1)178_random_n10_{}st'.format(nst) # Will be used for saving files.
    # Read synthetic steam from disk
    stm = read('save/Test_Stream_{}/{}_Trace_*'.format(no_of_event,no_of_event))

    # Process and trim the raw trace
    trim_start=5
    trim_end= -225
    stm_processed=EQScanner.signal_processing(stm=stm,
                                process=False,hp=5.0,lp=49.0, 
                                trim= True, trim_start= trim_start, trim_end= trim_end, 
                                plot=False)

    # For generating synthetic seismogram example (plot only 1 station)
    #stm_processed[:3].plot(size=(1200,400),outfile='save/figs/{}_syn_seis_ex.png'.format(no_of_event),format='png')

    for cf_type_l in [['CECM'],['ST*LT'], ['ST*LT','CECM'],['ST/LT'],['RP/LP'] ]:#['CECM'],['ST*LT'], ['ST*LT','CECM'], 
        cf_type_list =cf_type_l
        cf_type,cf_E,cf_N,cf_Z= EQScanner.chracteristic_function(
                                                    stm_processed, cf_type_list= cf_type_list, 
                                                    plot= True, plot_size=(10,14), save_fig=True,
                                                    save_cf= False, no_of_event=no_of_event,linewidth=1)
        
        #Cf_EN=np.sqrt(cf_E*cf_N)
        plt.close()
        time.sleep(1)

        # Parameters for scanning event origin time
        tmin,tmax = 23,26                     # Range of possible origin time (in sec after the start time)
        dt = 0.1                             # Step for scanning specific time (in sec)
        # Parameters for scanning the hypocenter
        zmin,zmax = src_z-6, src_z+6        # Range of possible depth (in km)
        xmin,xmax = src_x-60, src_x+60               # E-W boundaries of the scanning area 
                                             # (in Easting; omit the last 2 digits: ie. dx=100m)
        ymin,ymax = src_y-60, src_y+60             # S-N boundaries of the scanning area
                                             # (in Northing; omit the last 2 digits: ie. dy=100m)
        dx = 5                               # x interval (ex. dx = 5 then resolusion = 500m)
        dy = 5                               # y interval (ex. dy = 5 then resolusion = 500m)
        dz = 0.5                             # z interval (ex. dx = 0.5 then resolusion = 0.5km)
        utm_zone = (27,'V')                  # UTM zone number of the scanning area (ex. (27,'V))
        #method  = 'L1'                  # Option:'L2-Norm', 'EDT'
        method  = 'EDT'                     # Option:'L2-Norm', 'EDT'
        
        color_dict=  {'kri':'tab:blue','faf':'tab:orange','lat':'tab:green','sea':'tab:red',
                     'klv':'tab:purple','elb':'xkcd:sienna','ash':'tab:pink','iss':'tab:gray',
                     'hdv':'tab:olive','lsf':'tab:cyan','lhl':'xkcd:tomato','vog':'xkcd:teal',
                     'kas':'xkcd:green','blf':'xkcd:blue','bhr':'xkcd:orchid','hah':'xkcd:orangered',
                     'svh':'xkcd:khaki','has':'xkcd:grey'}
        
        # Scan to get the brightness matrices
        C_spatial, C_temporal    = EQScanner.SourceScan(   
                                    stm_processed= stm_processed, 
                                    Cf_Z=cf_Z, Cf_E=cf_E, Cf_N=cf_N, 
                                    tmin= tmin, tmax= tmax, dt= dt,
                                    zmin= zmin, zmax= zmax, dz= dz,
                                    xmin= xmin, xmax= xmax, dx= dx,
                                    ymin= ymin, ymax= ymax, dy= dy, 
                                    utm_zone= utm_zone, TTModel_1= TTModel_P, TTModel_2=TTModel_S,
                                    method=method, tmin_checker = False, 
                                    save_C= True, no_of_event= no_of_event, cf_type_list=cf_type_list,
                                    add_filename='_ScanPS',alarm=False)

        C_s= C_spatial
        time.sleep(1)

        # Analyse to get the maximun brightness
        C_xy, C_zy, C_zx, coor_max_Cxy, coor_max_Czy, coor_max_Czx, imax_Cs= EQScanner.analyse_spatial(
                                    stm_processed= stm_processed, C_s= C_s,
                                    zmin= zmin, zmax= zmax, dz= dz,
                                    xmin= xmin, xmax= xmax, dx= dx,
                                    ymin= ymin, ymax= ymax, dy= dy)

        t_eo= EQScanner.analyse_temporal(
            stm_processed= stm_processed, C_temporal= C_temporal, imax_Cs=imax_Cs, tmin=tmin)

        # Plot the Source Scanning result
        EQScanner.source_plot(    stm_processed=stm_processed,
                        C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                        coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                        dt= dt, method=method,cf_type_list=cf_type_list,
                        zmin= zmin, zmax= zmax, dz= dz,
                        xmin= xmin, xmax= xmax, dx= dx,
                        ymin= ymin, ymax= ymax, dy= dy,
                        plot_mp = False,# mp_scale = 0.1, C_6D=C_6D,
                        draw_rect=False, 
                        #rect_x=[4465,4485], rect_y=[70850,70880], rect_z= [0,50],
                        plot_ref= '--', figsize=[7,7.3], vmin=None, 
                        compare_dtdr=True, 
                        tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                        save_fig=True, no_of_event= no_of_event, add_figname='_ScanPS_nosmth')
        plt.close()

        # Smooth the brightness matrix
        C_s= EQScanner.smooth(      C_spatial,
                                    zmin= zmin, zmax= zmax, dz= dz,
                                    xmin= xmin, xmax= xmax, dx= dx,
                                    ymin= ymin, ymax= ymax, dy= dy)

        # Analyse to get the maximun brightness
        C_xy, C_zy, C_zx, coor_max_Cxy, coor_max_Czy, coor_max_Czx, imax_Cs= EQScanner.analyse_spatial(
                                    stm_processed= stm_processed, C_s= C_s,
                                    zmin= zmin, zmax= zmax, dz= dz,
                                    xmin= xmin, xmax= xmax, dx= dx,
                                    ymin= ymin, ymax= ymax, dy= dy)

        t_eo= EQScanner.analyse_temporal(
            stm_processed= stm_processed, C_temporal= C_temporal, imax_Cs=imax_Cs, tmin=tmin)

        # Compare SSA result with SAC reference
        diff_in_t, diff_in_s = EQScanner.compare(	stm_processed=stm_processed,
                            C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                            coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                            tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                            save = True, no_of_event=no_of_event, method=method, add_title='_ScanPS_',
                            cf_type=cf_type,first_cf_type='CECM', filename='Results_slowPS__n10_scanPS_random')
    
        # Plot the Source Scanning result
        EQScanner.source_plot(    stm_processed=stm_processed,
                        C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                        coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                        dt= dt, method=method,cf_type_list=cf_type_list,
                        zmin= zmin, zmax= zmax, dz= dz,
                        xmin= xmin, xmax= xmax, dx= dx,
                        ymin= ymin, ymax= ymax, dy= dy,
                        plot_mp = False, #, mp_scale=0.1, C_4D=C_4D,
                        compare_dtdr=True, 
                        tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                        draw_rect=False, 
                        #rect_x=[4465,4485], rect_y=[70850,70880], rect_z= [0,50],
                        plot_ref= '--', figsize=[7,7.3], vmin=None,
                        save_fig=True, no_of_event= no_of_event, add_figname='_ScanPS_smth')
        plt.close()
        time.sleep(1)
#winsound.Beep(523, 1000)
#winsound.Beep(587, 500)
#winsound.Beep(523, 500)

In [ ]:
importlib.reload(EQScanner)

In [ ]:
importlib.reload(trigger)